In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import os

os.environ["JAX_PLATFORMS"] = "proxy"
os.environ["JAX_BACKEND_TARGET"] = "grpc://127.0.0.1:29000"

In [7]:
import sys

sys.path.insert(0, '/scratch/git/tunix')
sys.path.insert(0, '/scratch/git/pathways-utils')

import tunix
print(tunix.__file__)

import pathwaysutils

pathwaysutils.__file__

/scratch/git/tunix/tunix/__init__.py


'/scratch/git/pathways-utils/pathwaysutils/__init__.py'

In [8]:
import pathwaysutils
import jax 
import pprint 

pathwaysutils.initialize()
pprint.pprint(jax.devices())

[device(0,TPU_DEVICE,coords=[0,0,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3),
 device(1,TPU_DEVICE,coords=[1,0,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3),
 device(2,TPU_DEVICE,coords=[0,1,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3),
 device(3,TPU_DEVICE,coords=[1,1,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3),
 device(4,TPU_DEVICE,coords=[0,0,1,0],vtask=1,slice=0,default_mem=device,mem_spaces=3),
 device(5,TPU_DEVICE,coords=[1,0,1,0],vtask=1,slice=0,default_mem=device,mem_spaces=3),
 device(6,TPU_DEVICE,coords=[0,1,1,0],vtask=1,slice=0,default_mem=device,mem_spaces=3),
 device(7,TPU_DEVICE,coords=[1,1,1,0],vtask=1,slice=0,default_mem=device,mem_spaces=3)]


In [9]:
import logging
import sys

# Remove existing handlers to prevent duplicate logs or conflicts
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    stream=sys.stdout,  # Direct logs to standard output (notebook cell)
    level=logging.INFO, # Set the minimum level to INFO
    format="%(asctime)s - %(levelname)s - %(message)s", # Optional: customize the format
    datefmt="%Y-%m-%d %H:%M:%S" # Optional: customize the date format
)

In [ ]:
import jax
jax.devices()

[device(0,TPU_DEVICE,coords=[0,0,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3),
 device(1,TPU_DEVICE,coords=[1,0,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3),
 device(2,TPU_DEVICE,coords=[0,1,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3),
 device(3,TPU_DEVICE,coords=[1,1,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3),
 device(4,TPU_DEVICE,coords=[0,0,1,0],vtask=1,slice=0,default_mem=device,mem_spaces=3),
 device(5,TPU_DEVICE,coords=[1,0,1,0],vtask=1,slice=0,default_mem=device,mem_spaces=3),
 device(6,TPU_DEVICE,coords=[0,1,1,0],vtask=1,slice=0,default_mem=device,mem_spaces=3),
 device(7,TPU_DEVICE,coords=[1,1,1,0],vtask=1,slice=0,default_mem=device,mem_spaces=3)]

In [11]:
import importlib

if importlib.util.find_spec("dotenv") is None:
  print("Required packages not found. Running full installation...")
  %pip install -q dotenv
  %pip install -q kagglehub
  %pip install -q safetensors
  %pip install -q tensorflow
  %pip install -q tensorflow_datasets
  %pip install -q tensorboardX
  %pip install -q transformers
  %pip install -q grain
  %pip install -q datasets
  %pip install -q wandb
  %pip install -q git+https://github.com/jax-ml/jax
  %pip install -q git+https://github.com/google/tunix
  %pip install -q git+https://github.com/google/qwix
  %pip uninstall -q flax -y
  %pip install -q git+https://github.com/google/flax
  %pip install -q 'numpy>2'

In [ ]:
import contextlib
import os

from flax import nnx
import grain
import jax
from jax import numpy as jnp
import optax

import optax
from orbax import checkpoint as ocp

from tunix.models.qwen2 import params as params_lib
from tunix.models.qwen2 import model as model_lib
from tunix.sft import metrics_logger
from tunix.rl.agentic.agents import model_agent
from tunix.rl.agentic.environments import task_environment
from tunix.rl.agentic.rewards import reward
from tunix.rl.agentic.trajectory import trajectory_collect_engine
from tunix.rl.agentic.parser.chat_template_parser import parser
import jax
import numpy as np
from tunix.rl.experimental.agentic_grpo_learner import GRPOConfig, GRPOLearner
from tunix.rl import rl_cluster as rl_cluster_lib
from tunix.rl.rollout import base_rollout
from tunix.sft import utils as sft_utils
from tunix.utils import math_rewards

In [13]:
# ====== Data ======
TRAIN_FRACTION = 1.0

# ====== Reproducibility ======
SEED = 42

# ====== LoRA ======
RANK = 64
ALPHA = 64.0
TRAIN_WITH_LORA = False

# ====== Sharding ======
MESH = [(2, 4), ("fsdp", "tp")]

# ====== GRPO ======
# === Generation during GRPO training ===
MAX_PROMPT_LENGTH = 2048
TOTAL_GENERATION_STEPS = 512
# Important to keep a high-ish temperature for varied, diverse responses during
# training.
TEMPERATURE = 0.6
TOP_P = 0.95
TOP_K = 50
# The number of times the policy generates multiple responses for a given prompt
# within a single training step. This corresponds to `G` in Algorithm 1 in the
# paper. The "group" in GRPO comes from here.
NUM_GENERATIONS = 2

# === other GRPO configs ===
# The number of iterations per batch (𝜇 in GRPO algo 1).
NUM_ITERATIONS = 1
# The coefficient for the KL divergence penalty (𝛽) in the GRPO loss function.
# Important to keep a high enough value for this, otherwise, the KL divergence
# can increase unchecked.
BETA = 0.001
# Epsilon value for clipping (𝜀 in GRPO loss in paper). Similar to PPO, for
# stable updates.
EPSILON = 0.2

# ====== Training ======
BATCH_SIZE = 16
MINI_BATCH_SIZE = 16
# ROLLOUT_MICRO_BATCH_SIZE = 8
# LOGPS_MICRO_BATCH_SIZE = 8
NUM_BATCHES = 100
# Keep `NUM_TEST_BATCHES` low so that evaluation runs quickly. It can be
# increased to a max. of 330 (if batch size is 4).
NUM_TEST_BATCHES = 50

EVAL_EVERY_N_STEPS = 1000  # this doesn't matter if `TRAIN_FRACTION = 1.0`.
NUM_EPOCHS = 100 # can potentially train for more epochs

# Number of training steps.
MAX_STEPS = int(NUM_BATCHES * NUM_ITERATIONS * TRAIN_FRACTION * NUM_EPOCHS)

# === AdamW, warmup, cosine scheduler ===
LEARNING_RATE = 1e-6
B1 = 0.9  # Adam beta1
B2 = 0.99  # Adam beta2
WEIGHT_DECAY = 0.1
# == Cosine decay with warmup scheduler ==
# Linearly increase learning rate from 0. to 5e-6 in the first 10% training
# steps, and then gradually decrease the learning rate to 0 using cosine
# scheduler.
WARMUP_STEPS = int(0.1 * MAX_STEPS)
# == Grad clipping ==
# Grad clipping to prevent large gradients. Found this
# important to keep KL divergence in check.
MAX_GRAD_NORM = 0.1

# ====== Checkpoint saving ======
SAVE_INTERVAL_STEPS = 500
MAX_TO_KEEP = 4
DO_MEM_PROFILING = False

# ====== Inference ======
GENERATION_CONFIGS = {
    # greedy search
    "greedy": {"temperature": 1e-4, "top_k": 1, "top_p": 1.0},
    # some randomness
    "standard": {"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    # liberal
    "liberal": {"temperature": 0.85, "top_k": 2000, "top_p": 1.0},
}
# ====== Rollout ======
ROLLOUT_ENGINE = "sglang_jax" # one of "vanilla", "vllm" or "sglang_jax"

In [14]:
try:
  from GOOGLE_INTERNAL_PACKAGE_PATH.pyglib import gfile
  file_open = gfile.Open

  NOTEBOOK_ENV = "g3"
except Exception:
  NOTEBOOK_ENV = "git"

  import fsspec

  file_open = fsspec.open

if NOTEBOOK_ENV == "g3":
  DATA_PATH_PREFIX = "/GOOGLE_INTERNAL_STOAGE_PATH/gg-d/home/qwix-dev/rl/data/"
  MODEL_PATH_PREFIX = "/GOOGLE_INTERNAL_STOAGE_PATH/gg-d/home/qwix-dev/"
  CKPT_DIR_PREFIX = "/GOOGLE_INTERNAL_STOAGE_PATH/gg-d/home/qwix-dev/"
else:
  DATA_PATH_PREFIX = "gs://tunix/rl/data"
  MODEL_PATH_PREFIX = "gs://tunix/rl/models"
  CKPT_DIR_PREFIX = "gs://tunix/rl/checkpoints"

print("NOTEBOOK_ENV: ", NOTEBOOK_ENV)
CKPT_DIR = os.path.join("/tmp/cp", "deepscaler_ckpt/01")

MODEL_VERSION = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
MODEL_PATH = os.path.join(MODEL_PATH_PREFIX, "DeepSeek-R1-Distill-Qwen-1.5B")

NOTEBOOK_ENV:  git


In [15]:
import pandas as pd
import datasets as datasets_lib
import transformers

Dataset = datasets_lib.Dataset
AutoTokenizer = transformers.AutoTokenizer


DEEPSCALER_DATA_PATH = os.path.join(os.getcwd(), "deepscaler.json")
AIME_2024_DATA_PATH = os.path.join(os.getcwd(), "train-00000-of-00001.parquet")

def create_datasets(
    train_ds_path: str = DEEPSCALER_DATA_PATH,
    test_ds_path: str = AIME_2024_DATA_PATH
):
  def preprocess_fn(example, index):
    return {
        "question": example["problem"],
        "ground_truth": example["answer"],
        "data_source": "math",
    }

  with file_open(train_ds_path) as train_f, file_open(test_ds_path, 'rb') as test_f:
    train_df = pd.read_json(train_f)
    test_df = pd.read_parquet(test_f)

  train_ds = Dataset.from_pandas(train_df).map(preprocess_fn, with_indices=True)
  test_ds = Dataset.from_pandas(test_df).map(preprocess_fn, with_indices=True)


  def process_item(item):
      question = item["question"]
      answer = item["answer"]

      instruction = "Let's think step by step, and put your final answer within \\boxed{}."
      prompt = f"{question} {instruction}"
      prompt = tokenizer.apply_chat_template(
          [{"role": "user", "content": prompt}],
          tokenize=False, add_generation_prompt=True)

      return {
          "prompts": prompt,
          "question": question,
          "answer": answer,
      }

  train_ds = grain.MapDataset.source(train_ds).map(process_item)
  test_ds = grain.MapDataset.source(test_ds).map(process_item)
  return train_ds, test_ds


2026-01-22 16:43:34 - INFO - TensorFlow version 2.20.0 available.
2026-01-22 16:43:34 - INFO - JAX version 0.8.1 available.


In [16]:
DEEPSCALER_DATA_PATH

'/mnt/disks/external_disk/git/tunix/examples/deepscaler/deepscaler.json'

In [17]:
tokenizer_source = MODEL_PATH if NOTEBOOK_ENV == "g3" else MODEL_VERSION
print(tokenizer_source)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_source)

chat_parser = parser.QwenChatTemplateParser(tokenizer)

deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B


In [18]:
train_dataset, test_dataset = create_datasets()

train_dataset = train_dataset.batch(BATCH_SIZE)[:NUM_BATCHES]
if TRAIN_FRACTION == 1.0:
  train_dataset = train_dataset.repeat(NUM_EPOCHS)
  val_dataset = None
else:
  train_dataset = train_dataset[: int(len(train_dataset) * TRAIN_FRACTION)]
  train_dataset = train_dataset.repeat(NUM_EPOCHS)
  val_dataset = train_dataset[int(len(train_dataset) * TRAIN_FRACTION) :].repeat(NUM_EPOCHS)
test_dataset = test_dataset.batch(BATCH_SIZE)[:NUM_TEST_BATCHES]

for s in iter(train_dataset):
  print(s)
  break

for s in iter(test_dataset):
  print(s)
  break

Map: 100%|██████████| 30/30 [00:00<00:00, 7457.87 examples/s]


{'answer': array(['-\\frac{2}{3}', '26', '1', '\\frac{\\sqrt{2}}{2}', '28', '25',
       '7', '\\frac{5}{3}', '80\\%', '10^{-x}', '7', '\\frac{1}{5}',
       '380', '20', '0', '\\frac{9}{10}'], dtype='<U18'), 'prompts': array(["<｜begin▁of▁sentence｜><｜User｜>The operation $\\otimes$ is defined for all nonzero numbers by $a \\otimes b = \\frac{a^{2}}{b}$. Determine $[(1 \\otimes 2) \\otimes 3] - [1 \\otimes (2 \\otimes 3)]$. Let's think step by step, and put your final answer within \\boxed{}.<｜Assistant｜><think>\n",
       "<｜begin▁of▁sentence｜><｜User｜>Doug constructs a square window using $8$ equal-size panes of glass. The ratio of the height to width for each pane is $5 : 2$, and the borders around and between the panes are $2$ inches wide. In inches, what is the side length of the square window? Let's think step by step, and put your final answer within \\boxed{}.<｜Assistant｜><think>\n",
       "<｜begin▁of▁sentence｜><｜User｜>Let $P(x)$ be a polynomial of degree $3n$ such that\n\\begin{

In [19]:
from jax.sharding import Mesh

devices = jax.devices()
split = int(len(devices) / 2)
rollout_devices = np.array(devices[:split-2]).reshape(split-2, 1)
train_devices = np.array(devices[split:]).reshape(split, 1)
rollout_mesh = Mesh(rollout_devices, axis_names=('fsdp', 'tp'))
train_mesh = Mesh(train_devices, axis_names=('fsdp', 'tp'))

config = model_lib.ModelConfig.deepseek_r1_distill_qwen_1p5b()
MODEL_PATH = "/tmp/models/DeepSeek-R1-Distill-Qwen-1.5B/DeepSeek-R1-Distill-Qwen-1.5B/"
print("MODEL_PATH: ", MODEL_PATH)
qwen2_ref = params_lib.create_model_from_safe_tensors(MODEL_PATH, config, train_mesh, dtype=jnp.float32)
sft_utils.show_hbm_usage()

MODEL_PATH:  /tmp/models/DeepSeek-R1-Distill-Qwen-1.5B/DeepSeek-R1-Distill-Qwen-1.5B/
2026-01-22 16:43:39 - INFO -  - Using Pathways compatible HBM stats collector
2026-01-22 16:43:39 - INFO - Using 0 Bytes on device(0,TPU_DEVICE,coords=[0,0,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 16:43:39 - INFO - Using 0 Bytes on device(1,TPU_DEVICE,coords=[1,0,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 16:43:39 - INFO - Using 0 Bytes on device(2,TPU_DEVICE,coords=[0,1,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 16:43:39 - INFO - Using 0 Bytes on device(3,TPU_DEVICE,coords=[1,1,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 16:43:39 - INFO - Using 1.7 GiB on device(4,TPU_DEVICE,coords=[0,0,1,0],vtask=1,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 16:43:39 - INFO - Using 1.7 GiB on device(5,TPU_DEVICE,coords=[1,0,1,0],vtask=1,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 16:43:39 - INFO - Using 1.7 GiB o

In [20]:
qwen2_actor = params_lib.create_model_from_safe_tensors(MODEL_PATH, config, train_mesh, dtype=jnp.float32)
sft_utils.show_hbm_usage()

2026-01-22 16:43:41 - INFO -  - Using Pathways compatible HBM stats collector
2026-01-22 16:43:41 - INFO - Using 0 Bytes on device(0,TPU_DEVICE,coords=[0,0,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 16:43:41 - INFO - Using 0 Bytes on device(1,TPU_DEVICE,coords=[1,0,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 16:43:41 - INFO - Using 0 Bytes on device(2,TPU_DEVICE,coords=[0,1,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 16:43:41 - INFO - Using 0 Bytes on device(3,TPU_DEVICE,coords=[1,1,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 16:43:41 - INFO - Using 3.3 GiB on device(4,TPU_DEVICE,coords=[0,0,1,0],vtask=1,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 16:43:41 - INFO - Using 3.3 GiB on device(5,TPU_DEVICE,coords=[1,0,1,0],vtask=1,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 16:43:41 - INFO - Using 3.3 GiB on device(6,TPU_DEVICE,coords=[0,1,1,0],vtask=1,slice=0,default_mem=device,mem_spaces=3

In [21]:
ModelAgent = model_agent.ModelAgent
TaskEnvironment = task_environment.TaskEnvironment
TrajectoryCollectEngine = trajectory_collect_engine.TrajectoryCollectEngine
checkpointing_options = ocp.CheckpointManagerOptions(
    save_interval_steps=SAVE_INTERVAL_STEPS, max_to_keep=MAX_TO_KEEP
)
metrics_logging_options = metrics_logger.MetricsLoggerOptions(
    log_dir="/tmp/tensorboard/grpo", flush_every_n_steps=20
)

optimizer = optax.adamw(
    learning_rate=optax.schedules.warmup_cosine_decay_schedule(
        init_value=0.0,
        peak_value=LEARNING_RATE,
        warmup_steps=WARMUP_STEPS,
        decay_steps=MAX_STEPS,
        end_value=0.0,
    ),
    b1=B1,
    b2=B2,
    weight_decay=WEIGHT_DECAY,
)
if MAX_GRAD_NORM is not None:
  optimizer = optax.chain(
      optax.clip_by_global_norm(max_norm=MAX_GRAD_NORM),
      optimizer,
  )
  
cluster_config = rl_cluster_lib.ClusterConfig(
    role_to_mesh={
        rl_cluster_lib.Role.ACTOR: train_mesh,
        rl_cluster_lib.Role.REFERENCE: train_mesh,
        rl_cluster_lib.Role.ROLLOUT: rollout_mesh,
    },
    rollout_engine=ROLLOUT_ENGINE,
    offload_to_cpu=False,
    training_config=rl_cluster_lib.RLTrainingConfig(
        actor_optimizer=optimizer,
        eval_every_n_steps=EVAL_EVERY_N_STEPS,
        max_steps=20,
        mini_batch_size=MINI_BATCH_SIZE,
        train_micro_batch_size = 1,  # larger than 1 will cause OOM on HBM
        # metrics logging
        metrics_logging_options=metrics_logging_options,
        # checkpoint saving
        checkpoint_root_directory=CKPT_DIR,
        checkpointing_options=checkpointing_options,
    ),
    rollout_config=base_rollout.RolloutConfig(
        max_tokens_to_generate=TOTAL_GENERATION_STEPS,
        max_prompt_length=MAX_PROMPT_LENGTH,
        kv_cache_size=MAX_PROMPT_LENGTH + TOTAL_GENERATION_STEPS + 256,
        temperature=TEMPERATURE,
        top_p=TOP_P,
        top_k=TOP_K,
        eos_tokens=[tokenizer.encode("<|im_end|>")[0]],
        # sglang-jax specific configs
        rollout_sglang_jax_model_version="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
        rollout_sglang_jax_mem_fraction_static=0.2,
        rollout_sglang_jax_init_with_random_weights=True,
        rollout_sglang_jax_disable_radix_cache=True,
        rollout_sglang_jax_enable_deterministic_sampling=False,
        rollout_sglang_jax_precompile_bs_paddings=[1, 2],
        rollout_sglang_jax_precompile_token_paddings=[2048, 4096, 8192],
        rollout_sglang_jax_chunked_prefill_size=2048,
        rollout_sglang_jax_page_size=64,
    ),
)

grpo_config = GRPOConfig(
    num_generations=NUM_GENERATIONS,
    num_iterations=NUM_ITERATIONS,
    beta=BETA,
    epsilon=EPSILON,
    system_prompt="",
    max_concurrency=8,
) 
  

In [22]:
rl_cluster = rl_cluster_lib.RLCluster(
    actor=qwen2_actor,
    reference=qwen2_ref,
    tokenizer=tokenizer,
    cluster_config=cluster_config,
)

wandb: WARNING The anonymous setting has no effect and will be removed in a future version.
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /home/tsbao_google_com/.netrc.
wandb: Currently logged in as: tsbao (tsbao-google) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


2026-01-22 16:43:43 - INFO - W&B run URL: https://wandb.ai/tsbao-google/tunix/runs/hw8ag7pa
2026-01-22 16:43:44 - INFO - server_args=ServerArgs(model_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', tokenizer_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, revision=None, model_impl='auto', model_layer_nums=None, host='127.0.0.1', port=30000, skip_server_warmup=False, warmups=None, dtype='auto', quantization=None, quantization_param_path=None, quantization_config_path=None, kv_cache_dtype='auto', mem_fraction_static=0.2, max_running_requests=None, max_total_tokens=None, max_prefill_tokens=16384, chunked_prefill_size=2048, enable_mixed_chunk=False, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=64, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='proxy', device_indexes=[0, 1], tp_siz

Loading Regular Weights: 100%|██████████| 367/367 [02:19<00:00,  2.63it/s]
Loading MoE Weights: 0it [00:00, ?it/s]
Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 4192.21it/s]


In [23]:
import logging
import sys

# Remove existing handlers to prevent duplicate logs or conflicts
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    stream=sys.stdout,  # Direct logs to standard output (notebook cell)
    level=logging.INFO, # Set the minimum level to INFO
    format="%(asctime)s - %(levelname)s - %(message)s", # Optional: customize the format
    datefmt="%Y-%m-%d %H:%M:%S" # Optional: customize the date format
)

sft_utils.show_hbm_usage()

2026-01-22 17:09:54 - INFO -  - Using Pathways compatible HBM stats collector
2026-01-22 17:09:54 - INFO - Using 16.7 GiB on device(0,TPU_DEVICE,coords=[0,0,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 17:09:54 - INFO - Using 16.7 GiB on device(1,TPU_DEVICE,coords=[1,0,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 17:09:54 - INFO - Using 0 Bytes on device(2,TPU_DEVICE,coords=[0,1,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 17:09:54 - INFO - Using 0 Bytes on device(3,TPU_DEVICE,coords=[1,1,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 17:09:54 - INFO - Using 11.6 GiB on device(4,TPU_DEVICE,coords=[0,0,1,0],vtask=1,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 17:09:54 - INFO - Using 11.6 GiB on device(5,TPU_DEVICE,coords=[1,0,1,0],vtask=1,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 17:09:54 - INFO - Using 11.6 GiB on device(6,TPU_DEVICE,coords=[0,1,1,0],vtask=1,slice=0,default_mem=device,mem_spa

In [ ]:
grpo_trainer = GRPOLearner(
    rl_cluster=rl_cluster,
    reward_fns=[
        math_rewards.math_reward,
    ],
    algo_config=grpo_config,
    chat_parser=chat_parser,
)
grpo_trainer.train(train_dataset)

2026-01-22 17:09:54 - INFO - AgenticRLLearner init - Using Pathways compatible HBM stats collector
2026-01-22 17:09:54 - INFO - Using 16.7 GiB on device(0,TPU_DEVICE,coords=[0,0,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 17:09:54 - INFO - Using 16.7 GiB on device(1,TPU_DEVICE,coords=[1,0,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 17:09:54 - INFO - Using 0 Bytes on device(2,TPU_DEVICE,coords=[0,1,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 17:09:54 - INFO - Using 0 Bytes on device(3,TPU_DEVICE,coords=[1,1,0,0],vtask=0,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 17:09:54 - INFO - Using 11.6 GiB on device(4,TPU_DEVICE,coords=[0,0,1,0],vtask=1,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 17:09:54 - INFO - Using 11.6 GiB on device(5,TPU_DEVICE,coords=[1,0,1,0],vtask=1,slice=0,default_mem=device,mem_spaces=3)
2026-01-22 17:09:54 - INFO - Using 11.6 GiB on device(6,TPU_DEVICE,coords=[0,1,1,0],vtask=1,slice=0,defau

Actor Training:   0%|          | 0/20 [00:00<?, ?step/s]/scratch/git/tunix/.venv/lib/python3.13/site-packages/jax/_src/core.py:1796: RuntimeWarning: coroutine method 'aclose' of 'TokenizerManager._handle_batch_request' was never awaited
  def get_aval(x: Any) -> Any:
/scratch/git/tunix/.venv/lib/python3.13/site-packages/jax/_src/interpreters/partial_eval.py:2815: RuntimeWarning: coroutine method 'aclose' of 'TokenizerManager._wait_one_response' was never awaited
  in_tracers = map(partial(inline_atom, src_), eqn.invars)


2026-01-22 17:17:11 - INFO - Decode batch. #running-req: 16, #token: 6400, token usage: 0.01, gen throughput (token/s): 12.38, #queue-req: 0, 
2026-01-22 17:17:58 - INFO - Decode batch. #running-req: 16, #token: 7168, token usage: 0.01, gen throughput (token/s): 13.44, #queue-req: 0, 
2026-01-22 17:18:38 - INFO - Train loop finished in: 118.3605 seconds
2026-01-22 17:18:43 - INFO - Training with mesh: Mesh('fsdp': 4, 'tp': 1, axis_types=(Auto, Auto)). Compiled train_step cache size: 1
2026-01-22 17:18:43 - INFO - Train loop finished in: 0.0860 seconds
2026-01-22 17:18:51 - INFO - Decode batch. #running-req: 16, #token: 7552, token usage: 0.01, gen throughput (token/s): 12.26, #queue-req: 0, 
2026-01-22 17:19:46 - INFO - Decode batch. #running-req: 15, #token: 7616, token usage: 0.01, gen throughput (token/s): 10.99, #queue-req: 0, 
2026-01-22 17:20:27 - INFO - Prefill batch. #new-seq: 12, #new-token: 1408, #cached-token: 0, token usage: 0.00, #running-req: 4, #queue-req: 0, 
2026-01-22

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir /tmp/tensorboard/grpo

Reusing TensorBoard on port 6006 (pid 195166), started 0:00:23 ago. (Use '!kill 195166' to kill it.)